# ===============================================================================
# ПОДГОТОВКА ДАННЫХ И ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ (EDA)
# ===============================================================================

**Цель:** Базовая подготовка данных для моделирования оттока клиентов

**Содержание:**
1. Загрузка и первичный анализ данных
2. EDA (Exploratory Data Analysis)
3. Временное разбиение (Train/Val/Test-OOT)
4. Базовый preprocessing
5. Разделение по сегментам (2 группы)
6. Correlation Analysis с target
7. Сохранение подготовленных данных

**Дата:** 2025-01-13  
**Random seed:** 42

# ===============================================================================

---
# 1. ИМПОРТ БИБЛИОТЕК И КОНФИГУРАЦИЯ

In [ ]:
# ====================================================================================
# ИМПОРТ БИБЛИОТЕК
# ====================================================================================

import os
import warnings
from datetime import datetime
from pathlib import Path
import time
import gc

# Данные
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pointbiserialr

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Настройки
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("="*80)
print("ПОДГОТОВКА ДАННЫХ И EDA")
print("="*80)
print(f"✓ Библиотеки импортированы")
print(f"  Pandas: {pd.__version__}")
print(f"  NumPy: {np.__version__}")
print(f"  Дата запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [ ]:
# ====================================================================================
# КОНФИГУРАЦИЯ
# ====================================================================================

class Config:
    """Централизованная конфигурация"""
    
    # ВОСПРОИЗВОДИМОСТЬ
    RANDOM_SEED = 42
    
    # ПУТИ
    DATA_DIR = Path("data")
    OUTPUT_DIR = Path("output")
    FIGURES_DIR = Path("figures")
    
    # ФАЙЛЫ
    TRAIN_FILE = "churn_train_ul.parquet"
    
    # КОЛОНКИ
    ID_COLUMNS = ['cli_code', 'client_id', 'observation_point']
    TARGET_COLUMN = 'target_churn_3m'
    SEGMENT_COLUMN = 'segment_group'
    DATE_COLUMN = 'observation_point'
    
    # СЕГМЕНТЫ (ДВЕ МОДЕЛИ)
    SEGMENT_1_NAME = "Small Business"
    SEGMENT_1_VALUES = ['SMALL_BUSINESS']
    
    SEGMENT_2_NAME = "Middle + Large Business"
    SEGMENT_2_VALUES = ['MIDDLE_BUSINESS', 'LARGE_BUSINESS']
    
    # ВРЕМЕННОЕ РАЗБИЕНИЕ
    TRAIN_SIZE = 0.70
    VAL_SIZE = 0.15
    TEST_SIZE = 0.15
    
    # CORRELATION ANALYSIS
    CORRELATION_P_VALUE_THRESHOLD = 0.05
    DATA_LEAKAGE_THRESHOLD = 0.9
    TOP_N_CORRELATIONS = 20
    TOP_N_VISUALIZATION = 30
    
    @classmethod
    def create_directories(cls):
        for dir_path in [cls.OUTPUT_DIR, cls.FIGURES_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    @classmethod
    def get_train_path(cls):
        return cls.DATA_DIR / cls.TRAIN_FILE

config = Config()
config.create_directories()
np.random.seed(config.RANDOM_SEED)

print("\n✓ Конфигурация инициализирована")
print(f"  Random seed: {config.RANDOM_SEED}")
print(f"  Сегмент 1: {config.SEGMENT_1_NAME} {config.SEGMENT_1_VALUES}")
print(f"  Сегмент 2: {config.SEGMENT_2_NAME} {config.SEGMENT_2_VALUES}")
print(f"  Split: {config.TRAIN_SIZE}/{config.VAL_SIZE}/{config.TEST_SIZE}")

---
# 2. ЗАГРУЗКА ДАННЫХ

In [ ]:
# ====================================================================================
# ЗАГРУЗКА ДАННЫХ
# ====================================================================================

train_path = config.get_train_path()

print("\n" + "="*80)
print("ЗАГРУЗКА ДАННЫХ")
print("="*80)
print(f"Файл: {train_path}")

if not train_path.exists():
    raise FileNotFoundError(
        f"Файл не найден: {train_path}\n"
        f"Убедитесь, что файл {config.TRAIN_FILE} находится в папке data/"
    )

file_size = train_path.stat().st_size / (1024**2)
print(f"Размер файла: {file_size:.2f} MB")

start = time.time()
df_full = pd.read_parquet(train_path)
load_time = time.time() - start

memory = df_full.memory_usage(deep=True).sum() / (1024**2)

print(f"\n✓ Загружено за {load_time:.2f} сек")
print(f"  Размер: {df_full.shape}")
print(f"  Память: {memory:.2f} MB")
print(f"  Строк: {len(df_full):,}")
print(f"  Колонок: {df_full.shape[1]}")
print("="*80)

---
# 3. EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
# ====================================================================================
# БАЗОВАЯ ИНФОРМАЦИЯ О ДАННЫХ
# ====================================================================================

print("\n" + "="*80)
print("БАЗОВАЯ ИНФОРМАЦИЯ О ДАННЫХ")
print("="*80)

print(f"\n1. РАЗМЕР ДАННЫХ:")
print(f"   Строк: {len(df_full):,}")
print(f"   Колонок: {df_full.shape[1]}")
print(f"   Память: {df_full.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

print(f"\n2. ТИПЫ ДАННЫХ:")
dtype_counts = df_full.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   {dtype}: {count}")

print(f"\n3. ПЕРВЫЕ 5 СТРОК:")
print(df_full.head())

In [ ]:
# ====================================================================================
# АНАЛИЗ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ
# ====================================================================================

print("\n" + "="*80)
print("АНАЛИЗ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ (TARGET)")
print("="*80)

# Общий churn rate
churn_rate = df_full[config.TARGET_COLUMN].mean()
n_churned = df_full[config.TARGET_COLUMN].sum()
n_total = len(df_full)
ratio = (1-churn_rate)/churn_rate

print(f"\n1. ОБЩИЙ CHURN RATE:")
print(f"   Churn rate: {churn_rate:.4f} ({churn_rate*100:.2f}%)")
print(f"   Churned: {n_churned:,}")
print(f"   Not churned: {n_total - n_churned:,}")
print(f"   Class ratio: 1:{ratio:.1f}")

# Churn rate по сегментам
print(f"\n2. CHURN RATE ПО СЕГМЕНТАМ:")
for segment in df_full[config.SEGMENT_COLUMN].unique():
    segment_df = df_full[df_full[config.SEGMENT_COLUMN] == segment]
    seg_churn = segment_df[config.TARGET_COLUMN].mean()
    seg_count = len(segment_df)
    seg_pct = seg_count / len(df_full) * 100
    print(f"   {segment}:")
    print(f"     Размер: {seg_count:,} ({seg_pct:.1f}%)")
    print(f"     Churn rate: {seg_churn:.4f} ({seg_churn*100:.2f}%)")

print("="*80)

In [ ]:
# ====================================================================================
# АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
# ====================================================================================

print("\n" + "="*80)
print("АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
print("="*80)

missing = df_full.isnull().sum()
missing_df = pd.DataFrame({
    'Missing': missing[missing > 0],
    'Percent': (missing[missing > 0] / len(df_full) * 100).round(2)
}).sort_values('Missing', ascending=False)

print(f"\nКолонок с пропусками: {len(missing_df)}")

if len(missing_df) > 0:
    print(f"\nТоп-20 колонок с наибольшим количеством пропусков:")
    print(missing_df.head(20))
else:
    print("\n✓ Пропущенных значений не обнаружено")

print("="*80)

In [ ]:
# ====================================================================================
# АНАЛИЗ КОНСТАНТНЫХ КОЛОНОК
# ====================================================================================

print("\n" + "="*80)
print("АНАЛИЗ КОНСТАНТНЫХ КОЛОНОК")
print("="*80)

constant_cols = []
for col in df_full.columns:
    if df_full[col].nunique(dropna=False) == 1:
        constant_cols.append(col)

print(f"\nКонстантных колонок: {len(constant_cols)}")

if constant_cols:
    print(f"\nСписок константных колонок:")
    for col in constant_cols:
        print(f"  - {col} (значение: {df_full[col].iloc[0]})")
else:
    print("\n✓ Константных колонок не обнаружено")

print("="*80)

In [ ]:
# ====================================================================================
# АНАЛИЗ ВРЕМЕННОГО РАСПРЕДЕЛЕНИЯ
# ====================================================================================

print("\n" + "="*80)
print("АНАЛИЗ ВРЕМЕННОГО РАСПРЕДЕЛЕНИЯ")
print("="*80)

# Конвертация даты
df_full[config.DATE_COLUMN] = pd.to_datetime(df_full[config.DATE_COLUMN])

print(f"\n1. ВРЕМЕННОЙ ПЕРИОД:")
print(f"   Начало: {df_full[config.DATE_COLUMN].min().date()}")
print(f"   Конец: {df_full[config.DATE_COLUMN].max().date()}")
print(f"   Уникальных дат: {df_full[config.DATE_COLUMN].nunique()}")

print(f"\n2. КЛИЕНТЫ:")
print(f"   Уникальных cli_code: {df_full['cli_code'].nunique():,}")
print(f"   Уникальных client_id: {df_full['client_id'].nunique():,}")

# Распределение по датам
print(f"\n3. РАСПРЕДЕЛЕНИЕ ЗАПИСЕЙ ПО ДАТАМ:")
date_dist = df_full.groupby(config.DATE_COLUMN).size()
print(f"   Среднее записей на дату: {date_dist.mean():.0f}")
print(f"   Минимум: {date_dist.min():,}")
print(f"   Максимум: {date_dist.max():,}")

print("="*80)

In [ ]:
# ====================================================================================
# ВИЗУАЛИЗАЦИЯ: РАСПРЕДЕЛЕНИЕ TARGET
# ====================================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Общее распределение
target_dist = df_full[config.TARGET_COLUMN].value_counts()
axes[0].bar(['No Churn', 'Churn'], [target_dist[0], target_dist[1]],
           color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[0].set_title('Общее распределение Target', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Количество')
axes[0].set_yscale('log')
for i, v in enumerate([target_dist[0], target_dist[1]]):
    axes[0].text(i, v, f'{v:,}\n({v/len(df_full)*100:.2f}%)',
                ha='center', va='bottom', fontsize=10)

# 2. По сегментам (stacked)
segment_churn = df_full.groupby([config.SEGMENT_COLUMN, 
                                  config.TARGET_COLUMN]).size().unstack(fill_value=0)
segment_churn.plot(kind='bar', stacked=True, ax=axes[1],
                  color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[1].set_title('Распределение по сегментам', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Сегмент')
axes[1].set_ylabel('Количество')
axes[1].legend(['No Churn', 'Churn'], loc='upper right')
axes[1].tick_params(axis='x', rotation=45)

# 3. Churn rate по сегментам
churn_rates = df_full.groupby(config.SEGMENT_COLUMN)[config.TARGET_COLUMN].mean() * 100
axes[2].bar(range(len(churn_rates)), churn_rates.values,
           color='coral', alpha=0.7, edgecolor='black')
axes[2].set_xticks(range(len(churn_rates)))
axes[2].set_xticklabels(churn_rates.index, rotation=45, ha='right')
axes[2].set_title('Churn Rate по сегментам', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Churn Rate (%)')
for i, v in enumerate(churn_rates.values):
    axes[2].text(i, v, f'{v:.2f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(config.FIGURES_DIR / 'eda_target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Сохранено: figures/eda_target_distribution.png")

---
# 4. ВРЕМЕННОЕ РАЗБИЕНИЕ (TEMPORAL SPLIT)

Разбиение данных по времени для предотвращения data leakage:  
- **Train:** 70% первых по времени
- **Validation:** 15%
- **Test (OOT):** 15% последних

In [ ]:
# ====================================================================================
# TEMPORAL SPLIT
# ====================================================================================

print("\n" + "="*80)
print("ВРЕМЕННОЕ РАЗБИЕНИЕ (TEMPORAL SPLIT)")
print("="*80)

# Сортировка по времени
df_sorted = df_full.sort_values(config.DATE_COLUMN).reset_index(drop=True)
unique_dates = sorted(df_sorted[config.DATE_COLUMN].unique())
n_dates = len(unique_dates)

print(f"\nУникальных дат: {n_dates}")
print(f"Период: {unique_dates[0].date()} - {unique_dates[-1].date()}")

# Cutoff indices
train_cutoff = int(n_dates * config.TRAIN_SIZE)
val_cutoff = int(n_dates * (config.TRAIN_SIZE + config.VAL_SIZE))

train_end = unique_dates[train_cutoff - 1]
val_end = unique_dates[val_cutoff - 1]

print(f"\nCutoff даты:")
print(f"  Train: до {train_end.date()} ({train_cutoff} дат)")
print(f"  Val: {unique_dates[train_cutoff].date()} - {val_end.date()} ({val_cutoff - train_cutoff} дат)")
print(f"  Test (OOT): {unique_dates[val_cutoff].date()}+ ({n_dates - val_cutoff} дат)")

# Создание split
train_df = df_sorted[df_sorted[config.DATE_COLUMN] <= train_end].copy()
val_df = df_sorted[(df_sorted[config.DATE_COLUMN] > train_end) & 
                   (df_sorted[config.DATE_COLUMN] <= val_end)].copy()
test_df = df_sorted[df_sorted[config.DATE_COLUMN] > val_end].copy()

# Статистика по split
print(f"\n{'='*80}")
print("СТАТИСТИКА ПО SPLIT")
print(f"{'='*80}")

for name, df in [('TRAIN', train_df), ('VALIDATION', val_df), ('TEST (OOT)', test_df)]:
    churn_r = df[config.TARGET_COLUMN].mean()
    print(f"\n{name}:")
    print(f"  Записей: {len(df):,}")
    print(f"  Клиентов (cli_code): {df['cli_code'].nunique():,}")
    print(f"  Период: {df[config.DATE_COLUMN].min().date()} - {df[config.DATE_COLUMN].max().date()}")
    print(f"  Churn rate: {churn_r:.4f} ({churn_r*100:.2f}%)")
    print(f"  Процент от общего: {len(df)/len(df_full)*100:.2f}%")

# Проверка data leakage
assert train_df[config.DATE_COLUMN].max() < val_df[config.DATE_COLUMN].min(), "Data leakage detected!"
assert val_df[config.DATE_COLUMN].max() < test_df[config.DATE_COLUMN].min(), "Data leakage detected!"
print(f"\n✓ Temporal ordering verified - NO DATA LEAKAGE")

print("="*80)

---
# 5. БАЗОВЫЙ PREPROCESSING

- Удаление константных колонок
- Заполнение пропусков (медиана для числовых, мода для категориальных)
- Удаление ID колонок

In [ ]:
# ====================================================================================
# УДАЛЕНИЕ КОНСТАНТНЫХ КОЛОНОК
# ====================================================================================

print("\n" + "="*80)
print("БАЗОВЫЙ PREPROCESSING")
print("="*80)

print("\n1. УДАЛЕНИЕ КОНСТАНТНЫХ КОЛОНОК")
print("-" * 80)

# Определяем константные колонки на train
constant_cols_train = []
for col in train_df.columns:
    if col in config.ID_COLUMNS + [config.TARGET_COLUMN]:
        continue
    if train_df[col].nunique(dropna=False) == 1:
        constant_cols_train.append(col)

print(f"Константных колонок на train: {len(constant_cols_train)}")

if constant_cols_train:
    print(f"\nУдаляемые колонки:")
    for col in constant_cols_train:
        print(f"  - {col}")
    
    # Удаляем из всех датасетов
    train_df = train_df.drop(columns=constant_cols_train)
    val_df = val_df.drop(columns=constant_cols_train)
    test_df = test_df.drop(columns=constant_cols_train)
    
    print(f"\n✓ Удалено {len(constant_cols_train)} константных колонок")
else:
    print("\n✓ Константных колонок не обнаружено")

print(f"\nРазмеры после удаления константных колонок:")
print(f"  Train: {train_df.shape}")
print(f"  Val: {val_df.shape}")
print(f"  Test: {test_df.shape}")

In [ ]:
# ====================================================================================
# ЗАПОЛНЕНИЕ ПРОПУСКОВ
# ====================================================================================

print("\n2. ЗАПОЛНЕНИЕ ПРОПУСКОВ")
print("-" * 80)

# Определяем колонки с пропусками на train
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in config.ID_COLUMNS + [config.TARGET_COLUMN]]

categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols = [c for c in categorical_cols if c not in config.ID_COLUMNS + [config.TARGET_COLUMN]]

# Считаем медианы и моды на train
numeric_fillvalues = {}
for col in numeric_cols:
    if train_df[col].isnull().any():
        numeric_fillvalues[col] = train_df[col].median()

categorical_fillvalues = {}
for col in categorical_cols:
    if train_df[col].isnull().any():
        mode_val = train_df[col].mode()
        categorical_fillvalues[col] = mode_val[0] if len(mode_val) > 0 else 'MISSING'

print(f"\nЧисловых колонок с пропусками: {len(numeric_fillvalues)}")
print(f"Категориальных колонок с пропусками: {len(categorical_fillvalues)}")

# Применяем заполнение
if numeric_fillvalues:
    for df in [train_df, val_df, test_df]:
        for col, fill_val in numeric_fillvalues.items():
            df[col] = df[col].fillna(fill_val)
    print(f"\n✓ Заполнено {len(numeric_fillvalues)} числовых колонок медианой")

if categorical_fillvalues:
    for df in [train_df, val_df, test_df]:
        for col, fill_val in categorical_fillvalues.items():
            df[col] = df[col].fillna(fill_val)
    print(f"✓ Заполнено {len(categorical_fillvalues)} категориальных колонок модой")

if not numeric_fillvalues and not categorical_fillvalues:
    print("\n✓ Пропусков не обнаружено")

# Проверка
train_missing = train_df.isnull().sum().sum()
val_missing = val_df.isnull().sum().sum()
test_missing = test_df.isnull().sum().sum()

print(f"\nПроверка пропусков после заполнения:")
print(f"  Train: {train_missing}")
print(f"  Val: {val_missing}")
print(f"  Test: {test_missing}")

In [ ]:
# ====================================================================================
# УДАЛЕНИЕ ID КОЛОНОК
# ====================================================================================

print("\n3. УДАЛЕНИЕ ID КОЛОНОК")
print("-" * 80)

print(f"\nУдаляемые колонки: {config.ID_COLUMNS}")

train_df = train_df.drop(columns=config.ID_COLUMNS)
val_df = val_df.drop(columns=config.ID_COLUMNS)
test_df = test_df.drop(columns=config.ID_COLUMNS)

print(f"\n✓ Удалено {len(config.ID_COLUMNS)} ID колонок")

print(f"\nИтоговые размеры после preprocessing:")
print(f"  Train: {train_df.shape}")
print(f"  Val: {val_df.shape}")
print(f"  Test: {test_df.shape}")

print("\n" + "="*80)

---
# 6. РАЗДЕЛЕНИЕ ПО СЕГМЕНТАМ

Создание двух групп данных:  
- **Segment 1:** Small Business только
- **Segment 2:** Middle Business + Large Business

**ВАЖНО:**  
- Для Segment 1 удаляем `segment_group` (там одно значение - не информативно)
- Для Segment 2 оставляем `segment_group` (там два значения - полезная информация)

In [ ]:
# ====================================================================================
# РАЗДЕЛЕНИЕ ПО СЕГМЕНТАМ
# ====================================================================================

print("\n" + "="*80)
print("РАЗДЕЛЕНИЕ ПО СЕГМЕНТАМ")
print("="*80)

# SEGMENT 1: Small Business
print(f"\n1. SEGMENT 1: {config.SEGMENT_1_NAME.upper()}")
print("-" * 80)

seg1_train = train_df[train_df[config.SEGMENT_COLUMN].isin(config.SEGMENT_1_VALUES)].copy()
seg1_val = val_df[val_df[config.SEGMENT_COLUMN].isin(config.SEGMENT_1_VALUES)].copy()
seg1_test = test_df[test_df[config.SEGMENT_COLUMN].isin(config.SEGMENT_1_VALUES)].copy()

print(f"Исходные размеры:")
print(f"  Train: {seg1_train.shape}")
print(f"  Val: {seg1_val.shape}")
print(f"  Test: {seg1_test.shape}")

# Удаляем segment_group для seg1 (там одно значение)
print(f"\nУникальных значений segment_group: {seg1_train[config.SEGMENT_COLUMN].nunique()}")
print(f"Значения: {seg1_train[config.SEGMENT_COLUMN].unique()}")
print(f"\n→ Удаляем segment_group (не информативна для seg1)")

seg1_train = seg1_train.drop(columns=[config.SEGMENT_COLUMN])
seg1_val = seg1_val.drop(columns=[config.SEGMENT_COLUMN])
seg1_test = seg1_test.drop(columns=[config.SEGMENT_COLUMN])

print(f"\nИтоговые размеры seg1:")
print(f"  Train: {seg1_train.shape} | Churn: {seg1_train[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Val: {seg1_val.shape} | Churn: {seg1_val[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Test: {seg1_test.shape} | Churn: {seg1_test[config.TARGET_COLUMN].mean()*100:.2f}%")

# SEGMENT 2: Middle + Large Business
print(f"\n\n2. SEGMENT 2: {config.SEGMENT_2_NAME.upper()}")
print("-" * 80)

seg2_train = train_df[train_df[config.SEGMENT_COLUMN].isin(config.SEGMENT_2_VALUES)].copy()
seg2_val = val_df[val_df[config.SEGMENT_COLUMN].isin(config.SEGMENT_2_VALUES)].copy()
seg2_test = test_df[test_df[config.SEGMENT_COLUMN].isin(config.SEGMENT_2_VALUES)].copy()

print(f"Исходные размеры:")
print(f"  Train: {seg2_train.shape}")
print(f"  Val: {seg2_val.shape}")
print(f"  Test: {seg2_test.shape}")

# Оставляем segment_group для seg2 (там два значения)
print(f"\nУникальных значений segment_group: {seg2_train[config.SEGMENT_COLUMN].nunique()}")
print(f"Значения: {seg2_train[config.SEGMENT_COLUMN].unique()}")
print(f"\n→ Оставляем segment_group (полезный признак для seg2)")

print(f"\nИтоговые размеры seg2:")
print(f"  Train: {seg2_train.shape} | Churn: {seg2_train[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Val: {seg2_val.shape} | Churn: {seg2_val[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Test: {seg2_test.shape} | Churn: {seg2_test[config.TARGET_COLUMN].mean()*100:.2f}%")

print("\n" + "="*80)

---
# 7. CORRELATION ANALYSIS С TARGET

Анализ корреляции всех числовых признаков с целевой переменной используя **Point-Biserial Correlation**.  
Выполняется для обеих групп отдельно.

In [ ]:
# ====================================================================================
# ФУНКЦИИ ДЛЯ CORRELATION ANALYSIS
# ====================================================================================

def calculate_pointbiserial_correlations(df, target_col, p_threshold=0.05):
    """
    Рассчитывает Point-Biserial корреляцию для всех числовых признаков с бинарным target.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Датафрейм с данными
    target_col : str
        Название целевой колонки (бинарной)
    p_threshold : float
        Порог p-value для определения значимости
    
    Returns:
    --------
    pd.DataFrame
        Таблица с корреляциями, p-value и флагом значимости
    """
    
    # Получаем числовые колонки (кроме target)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c != target_col]
    
    results = []
    target_values = df[target_col].values
    
    print(f"Анализируем {len(numeric_cols)} числовых признаков...")
    
    for col in numeric_cols:
        feature_values = df[col].values
        
        # Пропускаем константные колонки
        if len(np.unique(feature_values)) == 1:
            continue
        
        try:
            # Point-Biserial correlation
            corr, pval = pointbiserialr(target_values, feature_values)
            
            results.append({
                'feature': col,
                'correlation': corr,
                'abs_correlation': abs(corr),
                'p_value': pval,
                'significant': pval < p_threshold
            })
        except Exception as e:
            print(f"  Ошибка для {col}: {e}")
            continue
    
    # Создаем DataFrame
    corr_df = pd.DataFrame(results)
    
    # Сортируем по модулю корреляции
    corr_df = corr_df.sort_values('abs_correlation', ascending=False).reset_index(drop=True)
    
    return corr_df


def plot_top_correlations(corr_df, segment_name, top_n=30, save_path=None):
    """
    Визуализация топ-N корреляций.
    
    Parameters:
    -----------
    corr_df : pd.DataFrame
        Таблица с корреляциями
    segment_name : str
        Название сегмента для заголовка
    top_n : int
        Количество топ корреляций для отображения
    save_path : Path or None
        Путь для сохранения графика
    """
    
    top_corr = corr_df.head(top_n).copy()
    
    fig, ax = plt.subplots(figsize=(12, max(8, top_n * 0.3)))
    
    # Цвета: положительная - зеленый, отрицательная - красный
    colors = ['green' if x >= 0 else 'red' for x in top_corr['correlation']]
    
    # Barplot
    bars = ax.barh(range(len(top_corr)), top_corr['correlation'].values,
                   color=colors, alpha=0.7, edgecolor='black')
    
    # Настройки
    ax.set_yticks(range(len(top_corr)))
    ax.set_yticklabels(top_corr['feature'].values, fontsize=9)
    ax.set_xlabel('Point-Biserial Correlation', fontsize=12, fontweight='bold')
    ax.set_title(f'Top-{top_n} Correlations with Target: {segment_name}',
                fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax.grid(axis='x', alpha=0.3)
    
    # Инвертируем ось Y чтобы самая высокая корреляция была сверху
    ax.invert_yaxis()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✓ Сохранено: {save_path}")
    
    plt.show()

print("✓ Функции для correlation analysis определены")

In [ ]:
# ====================================================================================
# CORRELATION ANALYSIS: SEGMENT 1
# ====================================================================================

print("\n" + "="*80)
print(f"CORRELATION ANALYSIS: SEGMENT 1 - {config.SEGMENT_1_NAME.upper()}")
print("="*80)

# Расчет корреляций на train
print(f"\nРасчет Point-Biserial корреляций на train данных...")
start_time = time.time()

corr_seg1 = calculate_pointbiserial_correlations(
    seg1_train, 
    config.TARGET_COLUMN,
    config.CORRELATION_P_VALUE_THRESHOLD
)

elapsed = time.time() - start_time
print(f"\n✓ Расчет завершен за {elapsed:.2f} сек")

# Статистика
print(f"\nОБЩАЯ СТАТИСТИКА:")
print(f"  Всего признаков: {len(corr_seg1)}")
print(f"  Значимых (p<0.05): {corr_seg1['significant'].sum()}")
print(f"  Средняя |корреляция|: {corr_seg1['abs_correlation'].mean():.4f}")
print(f"  Максимальная |корреляция|: {corr_seg1['abs_correlation'].max():.4f}")

# Проверка на data leakage
leakage_features = corr_seg1[corr_seg1['abs_correlation'] > config.DATA_LEAKAGE_THRESHOLD]
if len(leakage_features) > 0:
    print(f"\n⚠️  ВНИМАНИЕ: Обнаружены признаки с очень высокой корреляцией (>0.9):")
    print(leakage_features[['feature', 'correlation', 'p_value']].head(10))
    print(f"\n→ Это может указывать на data leakage! Проверьте эти признаки.")
else:
    print(f"\n✓ Признаков с подозрением на data leakage не обнаружено")

# Топ-20 корреляций
print(f"\nТОП-{config.TOP_N_CORRELATIONS} КОРРЕЛЯЦИЙ (по модулю):")
print(corr_seg1.head(config.TOP_N_CORRELATIONS)[['feature', 'correlation', 'p_value', 'significant']].to_string(index=False))

print("\n" + "="*80)

In [ ]:
# ====================================================================================
# ВИЗУАЛИЗАЦИЯ: CORRELATION SEGMENT 1
# ====================================================================================

plot_top_correlations(
    corr_seg1,
    config.SEGMENT_1_NAME,
    top_n=config.TOP_N_VISUALIZATION,
    save_path=config.FIGURES_DIR / 'correlation_segment1.png'
)

In [ ]:
# ====================================================================================
# CORRELATION ANALYSIS: SEGMENT 2
# ====================================================================================

print("\n" + "="*80)
print(f"CORRELATION ANALYSIS: SEGMENT 2 - {config.SEGMENT_2_NAME.upper()}")
print("="*80)

# Расчет корреляций на train
print(f"\nРасчет Point-Biserial корреляций на train данных...")
start_time = time.time()

corr_seg2 = calculate_pointbiserial_correlations(
    seg2_train, 
    config.TARGET_COLUMN,
    config.CORRELATION_P_VALUE_THRESHOLD
)

elapsed = time.time() - start_time
print(f"\n✓ Расчет завершен за {elapsed:.2f} сек")

# Статистика
print(f"\nОБЩАЯ СТАТИСТИКА:")
print(f"  Всего признаков: {len(corr_seg2)}")
print(f"  Значимых (p<0.05): {corr_seg2['significant'].sum()}")
print(f"  Средняя |корреляция|: {corr_seg2['abs_correlation'].mean():.4f}")
print(f"  Максимальная |корреляция|: {corr_seg2['abs_correlation'].max():.4f}")

# Проверка на data leakage
leakage_features = corr_seg2[corr_seg2['abs_correlation'] > config.DATA_LEAKAGE_THRESHOLD]
if len(leakage_features) > 0:
    print(f"\n⚠️  ВНИМАНИЕ: Обнаружены признаки с очень высокой корреляцией (>0.9):")
    print(leakage_features[['feature', 'correlation', 'p_value']].head(10))
    print(f"\n→ Это может указывать на data leakage! Проверьте эти признаки.")
else:
    print(f"\n✓ Признаков с подозрением на data leakage не обнаружено")

# Топ-20 корреляций
print(f"\nТОП-{config.TOP_N_CORRELATIONS} КОРРЕЛЯЦИЙ (по модулю):")
print(corr_seg2.head(config.TOP_N_CORRELATIONS)[['feature', 'correlation', 'p_value', 'significant']].to_string(index=False))

print("\n" + "="*80)

In [ ]:
# ====================================================================================
# ВИЗУАЛИЗАЦИЯ: CORRELATION SEGMENT 2
# ====================================================================================

plot_top_correlations(
    corr_seg2,
    config.SEGMENT_2_NAME,
    top_n=config.TOP_N_VISUALIZATION,
    save_path=config.FIGURES_DIR / 'correlation_segment2.png'
)

---
# 8. СОХРАНЕНИЕ ПОДГОТОВЛЕННЫХ ДАННЫХ

Сохраняем все подготовленные данные и результаты анализа:

In [ ]:
# ====================================================================================
# СОХРАНЕНИЕ ДАННЫХ
# ====================================================================================

print("\n" + "="*80)
print("СОХРАНЕНИЕ ПОДГОТОВЛЕННЫХ ДАННЫХ")
print("="*80)

start_time = time.time()

# SEGMENT 1 DATA
print("\n1. Сохранение данных Segment 1...")
seg1_train.to_parquet(config.OUTPUT_DIR / 'seg1_train.parquet', index=False)
seg1_val.to_parquet(config.OUTPUT_DIR / 'seg1_val.parquet', index=False)
seg1_test.to_parquet(config.OUTPUT_DIR / 'seg1_test.parquet', index=False)
print(f"   ✓ seg1_train.parquet ({seg1_train.shape})")
print(f"   ✓ seg1_val.parquet ({seg1_val.shape})")
print(f"   ✓ seg1_test.parquet ({seg1_test.shape})")

# SEGMENT 2 DATA
print("\n2. Сохранение данных Segment 2...")
seg2_train.to_parquet(config.OUTPUT_DIR / 'seg2_train.parquet', index=False)
seg2_val.to_parquet(config.OUTPUT_DIR / 'seg2_val.parquet', index=False)
seg2_test.to_parquet(config.OUTPUT_DIR / 'seg2_test.parquet', index=False)
print(f"   ✓ seg2_train.parquet ({seg2_train.shape})")
print(f"   ✓ seg2_val.parquet ({seg2_val.shape})")
print(f"   ✓ seg2_test.parquet ({seg2_test.shape})")

# CORRELATION TABLES
print("\n3. Сохранение таблиц корреляций...")
corr_seg1.to_csv(config.OUTPUT_DIR / 'correlation_segment1.csv', index=False)
corr_seg2.to_csv(config.OUTPUT_DIR / 'correlation_segment2.csv', index=False)
print(f"   ✓ correlation_segment1.csv ({len(corr_seg1)} признаков)")
print(f"   ✓ correlation_segment2.csv ({len(corr_seg2)} признаков)")

elapsed = time.time() - start_time

print(f"\n✓ Все данные сохранены за {elapsed:.2f} сек")
print("\n" + "="*80)

In [ ]:
# ====================================================================================
# ИТОГОВАЯ СВОДКА
# ====================================================================================

print("\n" + "="*80)
print("✓✓✓ ПОДГОТОВКА ДАННЫХ И EDA ЗАВЕРШЕНЫ УСПЕШНО ✓✓✓")
print("="*80)

print(f"\nДата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Random seed: {config.RANDOM_SEED}")

print(f"\n{'='*80}")
print("СОХРАНЕННЫЕ ФАЙЛЫ")
print(f"{'='*80}")

print(f"\nDATA FILES (output/):")
print(f"  • seg1_train.parquet - {seg1_train.shape}")
print(f"  • seg1_val.parquet - {seg1_val.shape}")
print(f"  • seg1_test.parquet - {seg1_test.shape}")
print(f"  • seg2_train.parquet - {seg2_train.shape}")
print(f"  • seg2_val.parquet - {seg2_val.shape}")
print(f"  • seg2_test.parquet - {seg2_test.shape}")

print(f"\nCORRELATION ANALYSIS (output/):")
print(f"  • correlation_segment1.csv - {len(corr_seg1)} признаков")
print(f"  • correlation_segment2.csv - {len(corr_seg2)} признаков")

print(f"\nVISUALIZATIONS (figures/):")
print(f"  • eda_target_distribution.png")
print(f"  • correlation_segment1.png")
print(f"  • correlation_segment2.png")

print(f"\n{'='*80}")
print("КЛЮЧЕВЫЕ СТАТИСТИКИ")
print(f"{'='*80}")

print(f"\nSEGMENT 1: {config.SEGMENT_1_NAME}")
print(f"  Train: {len(seg1_train):,} | Churn: {seg1_train[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Val: {len(seg1_val):,} | Churn: {seg1_val[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Test: {len(seg1_test):,} | Churn: {seg1_test[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Признаков: {seg1_train.shape[1] - 1} (без target)")
print(f"  Значимых корреляций: {corr_seg1['significant'].sum()}")

print(f"\nSEGMENT 2: {config.SEGMENT_2_NAME}")
print(f"  Train: {len(seg2_train):,} | Churn: {seg2_train[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Val: {len(seg2_val):,} | Churn: {seg2_val[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Test: {len(seg2_test):,} | Churn: {seg2_test[config.TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Признаков: {seg2_train.shape[1] - 1} (без target)")
print(f"  Значимых корреляций: {corr_seg2['significant'].sum()}")

print(f"\n{'='*80}")
print("СЛЕДУЮЩИЙ ШАГ: Feature Engineering и обучение моделей")
print(f"{'='*80}")